**Generating Elasticsearch Top-k Results for Re-ranking**

In [1]:
!pip3 install elasticsearch

In [2]:
!pip3 install elasticsearch-dsl

In [3]:
from elasticsearch import Elasticsearch

import sys
sys.path.insert(0, '../../../BERT-FAQ/')

# import required classes and functions
from reranker import ReRanker
from evaluation import get_relevance_label_df

# import utility functions
from shared.utils import load_from_json
from shared.utils import dump_to_json
from shared.utils import make_dirs

In [4]:
# Generate list of test queries, relevance labels for ReRanker class
query_answer_pair_filepath = '../../../BERT-FAQ/data/FAQIR/query_answer_pairs.json'
relevance_label_df = get_relevance_label_df(query_answer_pair_filepath)
test_queries = relevance_label_df[relevance_label_df['query_type'] == 'user_query'].question.unique()

In [5]:
# check first 5 occurences for test_queries
test_queries[:5]

array(['How to increase the efficiency of my car with respect to fuel expended?',
       'Is there a special way to drive to decreasse my fuel consumption?',
       'How to use less gas in my car?',
       'I want to save some money on gas. What can I do to make my car use less gas?',
       'Are there any simple tweaks one can do to a car to make it more fuel efficient?'],
      dtype=object)

In [6]:
test_queries = list(test_queries)

In [7]:
# check relevance_label_df
relevance_label_df

,answer,id,label,query_type,question
0,I have had good luck with commercial products ...,1,1,faq,"How to remove tree sap from car? Hi,I have som..."
1,Goo Gone is a fantastic product (available at ...,2,1,faq,How do I get a sticker off of a car window (p...
2,1. Inspect the black rubber cover that fits in...,3,1,faq,How to get rid of garbage disposal odor?
3,"To properly maintain your lawnmover, you'll ne...",4,1,faq,How to maintain a lawn mower?
4,Click on the link and follow the steps.,5,1,faq,How to fix vertical blinds? I can no longer re...
...,...,...,...,...,...
16319,"Get a bucket, fold some cotton towels into it,...",16320,1,user_query,Cigarette smoke odor is so annoying. Some frie...
16320,"Get a bucket, fold some cotton towels into it,...",16321,1,user_query,I love smoking a pipe but the smell is quite d...
16321,"Get a bucket, fold some cotton towels into it,...",16322,1,user_query,What are the best tips for getting rid of the ...
16322,"Get a bucket, fold some cotton towels into it,...",16323,1,user_query,The weed smell is really hard to get rid of. D...


In [8]:
# Define instance of ReRanker class
r = ReRanker(
    bert_model_path='', 
    test_queries=test_queries, relevance_label_df=relevance_label_df
)

In [9]:
# create output path to save Elasticsearch top-k results
output_path = "../../../BERT-FAQ/data/FAQIR/rank_results/unsupervised"
make_dirs(output_path)

In [10]:
# Select top-k value
top_k = 100

In [11]:
# Get top-k Elasticsearch results 

es = Elasticsearch([{'host':'localhost','port':9200}], http_auth=('elastic', 'elastic')) 

es_query_by_question = r.get_es_topk_results(es=es, index='faqir', query_by=['question'], top_k=top_k)
es_query_by_answer = r.get_es_topk_results(es=es, index='faqir', query_by=['answer'], top_k=top_k)
es_query_by_question_answer = r.get_es_topk_results(es=es, index='faqir', query_by=['question', 'answer'], top_k=top_k)
es_query_by_question_answer_concat = r.get_es_topk_results(es=es, index='faqir', query_by=['question_answer'], top_k=top_k)

2021-06-07 14:39:50 - Generating ES top-k results ...
  0%|          | 1/1183 [00:00<03:17,  5.98it/s]2021-06-07 14:39:50 - POST http://localhost:9200/faqir/_search [status:200 request:0.036s]
2021-06-07 14:39:50 - POST http://localhost:9200/faqir/_search [status:200 request:0.041s]
2021-06-07 14:39:50 - POST http://localhost:9200/faqir/_search [status:200 request:0.037s]
  0%|          | 4/1183 [00:00<02:33,  7.67it/s]2021-06-07 14:39:50 - POST http://localhost:9200/faqir/_search [status:200 request:0.043s]
2021-06-07 14:39:51 - POST http://localhost:9200/faqir/_search [status:200 request:0.034s]
2021-06-07 14:39:51 - POST http://localhost:9200/faqir/_search [status:200 request:0.043s]
  1%|          | 7/1183 [00:00<02:03,  9.53it/s]2021-06-07 14:39:51 - POST http://localhost:9200/faqir/_search [status:200 request:0.046s]
2021-06-07 14:39:51 - POST http://localhost:9200/faqir/_search [status:200 request:0.032s]
  1%|          | 9/1183 [00:00<01:44, 11.27it/s]2021-06-07 14:39:51 - POST

  6%|▌         | 73/1183 [00:04<01:08, 16.15it/s]2021-06-07 14:39:55 - POST http://localhost:9200/faqir/_search [status:200 request:0.132s]
2021-06-07 14:39:55 - POST http://localhost:9200/faqir/_search [status:200 request:0.050s]
  6%|▋         | 75/1183 [00:04<01:20, 13.81it/s]2021-06-07 14:39:55 - POST http://localhost:9200/faqir/_search [status:200 request:0.050s]
2021-06-07 14:39:55 - POST http://localhost:9200/faqir/_search [status:200 request:0.063s]
  7%|▋         | 77/1183 [00:04<01:16, 14.52it/s]2021-06-07 14:39:55 - POST http://localhost:9200/faqir/_search [status:200 request:0.065s]
2021-06-07 14:39:55 - POST http://localhost:9200/faqir/_search [status:200 request:0.075s]
  7%|▋         | 79/1183 [00:04<01:18, 14.07it/s]2021-06-07 14:39:55 - POST http://localhost:9200/faqir/_search [status:200 request:0.056s]
2021-06-07 14:39:55 - POST http://localhost:9200/faqir/_search [status:200 request:0.044s]
  7%|▋         | 81/1183 [00:04<01:13, 15.04it/s]2021-06-07 14:39:55 - POST 

 13%|█▎        | 149/1183 [00:07<00:39, 26.03it/s]2021-06-07 14:39:58 - POST http://localhost:9200/faqir/_search [status:200 request:0.033s]
2021-06-07 14:39:58 - POST http://localhost:9200/faqir/_search [status:200 request:0.030s]
2021-06-07 14:39:58 - POST http://localhost:9200/faqir/_search [status:200 request:0.055s]
 13%|█▎        | 152/1183 [00:07<00:40, 25.31it/s]2021-06-07 14:39:58 - POST http://localhost:9200/faqir/_search [status:200 request:0.034s]
2021-06-07 14:39:58 - POST http://localhost:9200/faqir/_search [status:200 request:0.037s]
2021-06-07 14:39:58 - POST http://localhost:9200/faqir/_search [status:200 request:0.032s]
 13%|█▎        | 155/1183 [00:08<00:40, 25.63it/s]2021-06-07 14:39:58 - POST http://localhost:9200/faqir/_search [status:200 request:0.040s]
2021-06-07 14:39:58 - POST http://localhost:9200/faqir/_search [status:200 request:0.035s]
2021-06-07 14:39:58 - POST http://localhost:9200/faqir/_search [status:200 request:0.035s]
 13%|█▎        | 158/1183 [00:0

 19%|█▉        | 225/1183 [00:11<00:43, 22.10it/s]2021-06-07 14:40:01 - POST http://localhost:9200/faqir/_search [status:200 request:0.046s]
2021-06-07 14:40:02 - POST http://localhost:9200/faqir/_search [status:200 request:0.033s]
2021-06-07 14:40:02 - POST http://localhost:9200/faqir/_search [status:200 request:0.033s]
 19%|█▉        | 228/1183 [00:11<00:41, 22.92it/s]2021-06-07 14:40:02 - POST http://localhost:9200/faqir/_search [status:200 request:0.030s]
2021-06-07 14:40:02 - POST http://localhost:9200/faqir/_search [status:200 request:0.027s]
2021-06-07 14:40:02 - POST http://localhost:9200/faqir/_search [status:200 request:0.043s]
 20%|█▉        | 231/1183 [00:11<00:39, 23.94it/s]2021-06-07 14:40:02 - POST http://localhost:9200/faqir/_search [status:200 request:0.040s]
2021-06-07 14:40:02 - POST http://localhost:9200/faqir/_search [status:200 request:0.047s]
2021-06-07 14:40:02 - POST http://localhost:9200/faqir/_search [status:200 request:0.056s]
 20%|█▉        | 234/1183 [00:1

 26%|██▌       | 303/1183 [00:14<00:32, 27.18it/s]2021-06-07 14:40:04 - POST http://localhost:9200/faqir/_search [status:200 request:0.042s]
2021-06-07 14:40:04 - POST http://localhost:9200/faqir/_search [status:200 request:0.041s]
2021-06-07 14:40:04 - POST http://localhost:9200/faqir/_search [status:200 request:0.037s]
 26%|██▌       | 306/1183 [00:14<00:34, 25.21it/s]2021-06-07 14:40:04 - POST http://localhost:9200/faqir/_search [status:200 request:0.044s]
2021-06-07 14:40:05 - POST http://localhost:9200/faqir/_search [status:200 request:0.034s]
2021-06-07 14:40:05 - POST http://localhost:9200/faqir/_search [status:200 request:0.089s]
 26%|██▌       | 309/1183 [00:14<00:40, 21.36it/s]2021-06-07 14:40:05 - POST http://localhost:9200/faqir/_search [status:200 request:0.040s]
2021-06-07 14:40:05 - POST http://localhost:9200/faqir/_search [status:200 request:0.068s]
2021-06-07 14:40:05 - POST http://localhost:9200/faqir/_search [status:200 request:0.037s]
 26%|██▋       | 312/1183 [00:1

...

2021-06-07 14:42:52 - POST http://localhost:9200/faqir/_search [status:200 request:0.033s]
 74%|███████▍  | 881/1183 [00:35<00:12, 24.86it/s]2021-06-07 14:42:52 - POST http://localhost:9200/faqir/_search [status:200 request:0.045s]
2021-06-07 14:42:52 - POST http://localhost:9200/faqir/_search [status:200 request:0.032s]
2021-06-07 14:42:52 - POST http://localhost:9200/faqir/_search [status:200 request:0.033s]
 75%|███████▍  | 884/1183 [00:35<00:12, 24.91it/s]2021-06-07 14:42:52 - POST http://localhost:9200/faqir/_search [status:200 request:0.029s]
2021-06-07 14:42:52 - POST http://localhost:9200/faqir/_search [status:200 request:0.026s]
2021-06-07 14:42:52 - POST http://localhost:9200/faqir/_search [status:200 request:0.030s]
2021-06-07 14:42:52 - POST http://localhost:9200/faqir/_search [status:200 request:0.049s]
 75%|███████▌  | 888/1183 [00:35<00:11, 25.55it/s]2021-06-07 14:42:52 - POST http://localhost:9200/faqir/_search [status:200 request:0.040s]
2021-06-07 14:42:52 - POST http

 81%|████████  | 957/1183 [00:38<00:09, 24.97it/s]2021-06-07 14:42:55 - POST http://localhost:9200/faqir/_search [status:200 request:0.038s]
2021-06-07 14:42:55 - POST http://localhost:9200/faqir/_search [status:200 request:0.036s]
2021-06-07 14:42:55 - POST http://localhost:9200/faqir/_search [status:200 request:0.029s]
 81%|████████  | 960/1183 [00:38<00:08, 25.54it/s]2021-06-07 14:42:55 - POST http://localhost:9200/faqir/_search [status:200 request:0.041s]
2021-06-07 14:42:55 - POST http://localhost:9200/faqir/_search [status:200 request:0.029s]
2021-06-07 14:42:55 - POST http://localhost:9200/faqir/_search [status:200 request:0.034s]
 81%|████████▏ | 963/1183 [00:38<00:08, 25.80it/s]2021-06-07 14:42:55 - POST http://localhost:9200/faqir/_search [status:200 request:0.028s]
2021-06-07 14:42:55 - POST http://localhost:9200/faqir/_search [status:200 request:0.029s]
2021-06-07 14:42:55 - POST http://localhost:9200/faqir/_search [status:200 request:0.040s]
 82%|████████▏ | 966/1183 [00:3

 87%|████████▋ | 1032/1183 [00:42<00:08, 17.20it/s]2021-06-07 14:42:59 - POST http://localhost:9200/faqir/_search [status:200 request:0.040s]
2021-06-07 14:42:59 - POST http://localhost:9200/faqir/_search [status:200 request:0.042s]
2021-06-07 14:42:59 - POST http://localhost:9200/faqir/_search [status:200 request:0.045s]
 87%|████████▋ | 1035/1183 [00:42<00:08, 18.31it/s]2021-06-07 14:42:59 - POST http://localhost:9200/faqir/_search [status:200 request:0.036s]
2021-06-07 14:42:59 - POST http://localhost:9200/faqir/_search [status:200 request:0.025s]
2021-06-07 14:42:59 - POST http://localhost:9200/faqir/_search [status:200 request:0.047s]
 88%|████████▊ | 1038/1183 [00:42<00:07, 19.91it/s]2021-06-07 14:42:59 - POST http://localhost:9200/faqir/_search [status:200 request:0.047s]
2021-06-07 14:43:00 - POST http://localhost:9200/faqir/_search [status:200 request:0.041s]
2021-06-07 14:43:00 - POST http://localhost:9200/faqir/_search [status:200 request:0.060s]
 88%|████████▊ | 1041/1183 [

2021-06-07 14:43:03 - POST http://localhost:9200/faqir/_search [status:200 request:0.031s]
2021-06-07 14:43:03 - POST http://localhost:9200/faqir/_search [status:200 request:0.042s]
 94%|█████████▎| 1108/1183 [00:46<00:03, 21.77it/s]2021-06-07 14:43:03 - POST http://localhost:9200/faqir/_search [status:200 request:0.049s]
2021-06-07 14:43:03 - POST http://localhost:9200/faqir/_search [status:200 request:0.041s]
2021-06-07 14:43:03 - POST http://localhost:9200/faqir/_search [status:200 request:0.036s]
 94%|█████████▍| 1111/1183 [00:46<00:03, 21.76it/s]2021-06-07 14:43:03 - POST http://localhost:9200/faqir/_search [status:200 request:0.041s]
2021-06-07 14:43:03 - POST http://localhost:9200/faqir/_search [status:200 request:0.043s]
2021-06-07 14:43:04 - POST http://localhost:9200/faqir/_search [status:200 request:0.035s]
 94%|█████████▍| 1114/1183 [00:47<00:03, 21.98it/s]2021-06-07 14:43:04 - POST http://localhost:9200/faqir/_search [status:200 request:0.035s]
2021-06-07 14:43:04 - POST h

100%|██████████| 1183/1183 [00:50<00:00, 23.57it/s]


In [12]:
# Save Elasticsearch results to json files
dump_to_json(es_query_by_question, output_path + '/es_query_by_question.json')
dump_to_json(es_query_by_answer, output_path + '/es_query_by_answer.json')
dump_to_json(es_query_by_question_answer, output_path + '/es_query_by_question_answer.json')
dump_to_json(es_query_by_question_answer_concat, output_path + '/es_query_by_question_answer_concat.json')